In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# URL of the Federal Reserve Calendar
url = 'https://www.federalreserve.gov/newsevents/calendar.htm'

# Send a GET request to fetch the raw HTML content
response = requests.get(url)
response.raise_for_status()  # Raise an exception for HTTP errors

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# **Note:** The following selectors are hypothetical.
# Inspect the actual page to determine the correct selectors.

# Example: Find all event containers
# Suppose each event is within a div with class 'event-item'
event_containers = soup.find_all('div', class_='event-item')

timestamps = []

for event in event_containers:
    # Find the <p> tag with the timestamp
    timestamp_tag = event.find('p', attrs={'ng-bind': 'value2.time'})
    if timestamp_tag:
        timestamp = timestamp_tag.get_text(strip=True)
        timestamps.append(timestamp)

# Print the extracted timestamps
print("Extracted Timestamps:")
for ts in timestamps:
    print(ts)

# **Optional:** Save to CSV
with open('federal_reserve_speech_timestamps.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Timestamp'])
    for ts in timestamps:
        writer.writerow([ts])

print("\nTimestamps have been saved to 'federal_reserve_speech_timestamps.csv'.")




Extracted Timestamps:

Timestamps have been saved to 'federal_reserve_speech_timestamps.csv'.


In [2]:
import requests

url = 'https://www.federalreserve.gov/newsevents/calendar.htm'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    page_content = response.text
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")
    page_content = ''


In [3]:
# Simple check to see if a known timestamp exists
sample_timestamp = "10:00 a.m."
if sample_timestamp in page_content:
    print("Timestamp found in static HTML.")
else:
    print("Timestamp not found in static HTML.")


Timestamp not found in static HTML.


In [20]:
import pandas as pd
dates = pd.read_csv("federal_reserve_speech_events_2024.csv")
speeches = pd.read_csv("filtered_speeches.csv")
dates

,Title,Start Time
0,Bank Regulation,2024-01-01 13:00:00-05:00
1,The Path Forward for Bank Capital Reform,2024-01-01 09:00:00-05:00
2,Cyber Risk,2024-01-01 09:00:00-05:00
3,Counterparty Credit Risk,2024-02-01 09:05:00-05:00
4,Economic Outlook,2024-02-01 19:35:00-05:00
...,...,...
108,Central Bank Independence and Economic Outlook,2024-11-01 07:00:00-04:00
109,Payments,2024-11-01 10:00:00-04:00
110,Conversation with Governor Bowman,2024-12-01 09:15:00-05:00
111,Moderated Discussion,2024-12-01 13:40:00-05:00


In [23]:
import pandas as pd

# Merge speeches with dates on the "Title" column (adjust for lowercase "title" in speeches)
merged = pd.merge(speeches, dates, left_on='title', right_on='Title', how='left')
print(merged.head())
# Update "Start Time" in speeches with the merged "Start Time"
speeches['Start Time'] = merged['Start Time_y']

# Since there's no "Duration" column, we'll only update "Start Time"
# If you need to compute additional columns, this is where you'd do it

# Display the updated DataFrame
print(speeches)

# Save to a new CSV file if needed
speeches.to_csv("updated_speeches.csv", index=False)
print("Updated speeches saved to 'updated_speeches.csv'")


         date                         speaker  \
0  2024-10-23     Governor Michelle W. Bowman   
1  2024-10-23     Governor Michelle W. Bowman   
2  2024-10-23     Governor Michelle W. Bowman   
3  2024-10-23     Governor Michelle W. Bowman   
4  2024-10-18  Governor Christopher J. Waller   

                                               title  \
0                                    Opening Remarks   
1                                    Opening Remarks   
2                                    Opening Remarks   
3                                    Opening Remarks   
4  Centralized and Decentralized Finance: Substit...   

                                     link  \
0  /newsevents/speech/bowman20241023a.htm   
1  /newsevents/speech/bowman20241023a.htm   
2  /newsevents/speech/bowman20241023a.htm   
3  /newsevents/speech/bowman20241023a.htm   
4  /newsevents/speech/waller20241018a.htm   

                                                text         Start Time_x  \
0  Good morning.1 It

In [30]:
data = pd.read_csv("updated_speeches.csv")
data["Start Time"] = pd.to_datetime(data["Start Time"])
data["End time"] = data["Start Time"] + pd.to_timedelta(data["Duration"], unit='m')
data


C:\Users\loren\AppData\Local\Temp\ipykernel_10756\2526841001.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data["Start Time"] = pd.to_datetime(data["Start Time"])
C:\Users\loren\AppData\Local\Temp\ipykernel_10756\2526841001.py:3: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  data["End time"] = data["Start Time"] + pd.to_timedelta(data["Duration"], unit='m')


,date,speaker,title,link,text,Start Time,Duration,End time
0,2024-10-23,Governor Michelle W. Bowman,Opening Remarks,/newsevents/speech/bowman20241023a.htm,Good morning.1 It is a pleasure to join you fo...,2024-04-01 09:05:00-04:00,7.0,2024-04-01 09:12:00-04:00
1,2024-10-18,Governor Christopher J. Waller,Centralized and Decentralized Finance: Substit...,/newsevents/speech/waller20241018a.htm,Thank you for inviting me to speak today.1 I h...,2024-06-01 03:00:00-04:00,16.0,2024-06-01 03:16:00-04:00
2,2024-10-14,Governor Christopher J. Waller,Thoughts on the Economy and Policy Rules at th...,/newsevents/speech/waller20241014a.htm,"Thank you, Athanasios, and thank you for the o...",2024-07-01 14:30:00-04:00,23.0,2024-07-01 14:53:00-04:00
3,2024-10-11,Governor Michelle W. Bowman,Challenges to the Community Banking Model,/newsevents/speech/bowman20241011a.htm,"Good afternoon, I'd like to begin by thanking ...",2024-10-01 09:00:00-04:00,17.0,2024-10-01 09:17:00-04:00
4,2024-10-10,Governor Lisa D. Cook,"Entrepreneurs, Innovation, and Participation",/newsevents/speech/cook20241010a.htm,"Thank you for the kind introduction, Jennet.1 ...",NaT,19.0,NaT
...,...,...,...,...,...,...,...,...
92,2024-02-02,Governor Michelle W. Bowman,The Future of Banking,/newsevents/speech/bowman20240202a.htm,Thank you for the invitation to join you today...,2024-04-01 09:05:00-04:00,26.0,2024-04-01 09:31:00-04:00
93,2024-01-17,Vice Chair for Supervision Michael S. Barr,Opening Remarks,/newsevents/speech/barr20240117a.htm,Thank you for the opportunity to welcome you t...,2024-06-01 03:00:00-04:00,6.0,2024-06-01 03:06:00-04:00
94,2024-01-17,Governor Michelle W. Bowman,The Path Forward for Bank Capital Reform,/newsevents/speech/bowman20240117a.htm,"""More is better."" This axiom often holds true ...",2024-07-01 14:30:00-04:00,22.0,2024-07-01 14:52:00-04:00
95,2024-01-16,Governor Christopher J. Waller,Almost as Good as It Gets…But Will It Last?,/newsevents/speech/waller20240116a.htm,"Thank you, David Wessel, and thank you to Broo...",2024-10-01 09:00:00-04:00,21.0,2024-10-01 09:21:00-04:00


In [17]:
from helpermodules2 import memory_handling as mh
import pandas as pd

data = mh.PickleHelper.pickle_load('SPY_2024_data').obj
data


,datetime,close,volume,timezone
0,2020-04-03 15:59:00,248.220000,4170390,America/New_York
1,2020-04-03 15:58:00,247.990010,1109892,America/New_York
2,2020-04-03 15:57:00,247.810000,1032571,America/New_York
3,2020-04-03 15:56:00,248.365010,764293,America/New_York
4,2020-04-03 15:55:00,248.059998,944028,America/New_York
...,...,...,...,...
425189,2024-12-17 09:34:00,604.039978,172420,America/New_York
425190,2024-12-17 09:33:00,604.049988,126522,America/New_York
425191,2024-12-17 09:32:00,604.210020,195426,America/New_York
425192,2024-12-17 09:31:00,603.950010,225427,America/New_York


In [14]:
from helpermodules2 import memory_handling as mh
import pandas as pd

data = mh.PickleHelper.pickle_load('SPY_2024_data').obj
data["datetime"] = pd.to_datetime(data["datetime"])  # Ensure 'datetime' is in datetime format
data["date"] = data["datetime"].dt.date  # Extract date part
data["time"] = data["datetime"].dt.time  # Extract time part


SPY_2024_data_updated=data
helper = mh.PickleHelper(SPY_2024_data_updated)
helper.pickle_dump("SPY_2024_data_updated")

In [15]:
data = mh.PickleHelper.pickle_load('SPY_2024_data_updated').obj
data

,datetime,close,volume,timezone,date,time
0,2024-01-11 15:59:00,476.329990,2251871,America/New_York,2024-01-11,15:59:00
1,2024-01-11 15:58:00,476.590000,569897,America/New_York,2024-01-11,15:58:00
2,2024-01-11 15:57:00,476.410000,354609,America/New_York,2024-01-11,15:57:00
3,2024-01-11 15:56:00,476.190000,289332,America/New_York,2024-01-11,15:56:00
4,2024-01-11 15:55:00,476.220000,514513,America/New_York,2024-01-11,15:55:00
...,...,...,...,...,...,...
86995,2024-12-16 09:34:00,605.890010,135872,America/New_York,2024-12-16,09:34:00
86996,2024-12-16 09:33:00,605.789980,77266,America/New_York,2024-12-16,09:33:00
86997,2024-12-16 09:32:00,605.859990,202285,America/New_York,2024-12-16,09:32:00
86998,2024-12-16 09:31:00,606.010010,132883,America/New_York,2024-12-16,09:31:00
